# RAG(Retrieval-Augmented-Generation) Client

## 1. Download Model

```bash
gai pull instructor-sentencepiece
```

## 2. Start RAG Service

* Go to /gai

* Open docker-compose.yml

* Make sure that the following section is not commented.

```yaml
    gai-rag-svr:
        image: kakkoii1337/gai-rag-svr:1.0.17
        container_name: gai-rag-svr
        ...
```

* Run the command:

```sh
docker-compose up
```



## Example 1: Ask about a Web Page

### Smoke Test

In [ ]:
from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync({
    "client_type":"gai",
    "url":"http://localhost:12036/gen/v1/rag"
    })
encode_response = await rag.encode_async("Hello, world!")
print(encode_response.encoded_data[:100])

[-0.0483919195830822, -0.0028134046588093042, -0.018624519929289818, -0.00446718605235219, 0.0265971552580595, 0.03294454514980316, 0.0061969878152012825, -0.008797473274171352, 0.004689408931881189, 0.06672601401805878, 0.0323917493224144, 0.011101982556283474, 0.025392407551407814, 0.043243832886219025, -0.024063099175691605, 0.006406883709132671, -0.06695400923490524, 0.008188264444470406, -0.04306166246533394, -0.014614514075219631, 0.04823515936732292, 0.009705739095807076, -0.006045014597475529, 0.03819381073117256, 0.018029479309916496, 0.040459856390953064, -0.046796754002571106, -0.0008777673356235027, 0.07563018053770065, -0.06293530762195587, 0.010833396576344967, -0.051085464656353, -0.02351493388414383, -0.026509391143918037, -0.03416185826063156, 0.025490660220384598, 0.027415992692112923, 0.04541882127523422, 0.02120743878185749, 0.028193600475788116, -0.00928844790905714, -0.006853214930742979, 0.015249678865075111, -0.03229036554694176, 0.020393043756484985, -0.0026365

### Step 1: Scrape text from the web for testing

Scrape a speech from the web and save it to a file.

In [3]:
import os
os.environ["LOG_LEVEL"]="INFO"
from gai.selenium.client import SeleniumClient

selenium_client = SeleniumClient(config={
    "type": "scraper",
    "engine": "selenium",
    "model": "selenium",
    "name": "gai-selenium",
    "client_type": "gai",
    "url": "http://localhost:12028/api/v1"
})

text,links = selenium_client.scrape("https://www.pmo.gov.sg/Newsroom/national-day-rally-2023")
import tempfile
with tempfile.NamedTemporaryFile(mode="w", delete=False) as f:
    f.write(text)
    file_path = f.name

/tmp/ipykernel_94074/2286610939.py:14: RuntimeWarning: coroutine 'SeleniumClient.scrape' was never awaited
  text,links = selenium_client.scrape("https://www.pmo.gov.sg/Newsroom/national-day-rally-2023")


TypeError: cannot unpack non-iterable coroutine object

### Step 2: Index the file

In [4]:
# Load RAG client
from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync(config={
    "client_type": "gai",
    "url": "http://localhost:12036/gen/v1/rag",
    "extra":{
        "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws"
    }
})

# Create callback
import json
async def async_callback(text):
    jsoned = json.loads(text)
    print(f"async_callback: {jsoned}")

# Index the sample speech
result = await rag.index_document_async(
    collection_name="demo",
    file_path=file_path,
    file_type="txt",
    title="2023 National Day Rally Speech",
    source="https://www.pmo.gov.sg/Newsroom/national-day-rally-2023",
    async_callback=async_callback,
)


async_callback: {'progress': 2}
async_callback: {'progress': 4}
async_callback: {'progress': 6}
async_callback: {'progress': 8}
async_callback: {'progress': 10}
async_callback: {'progress': 12}
async_callback: {'progress': 14}
async_callback: {'progress': 16}
async_callback: {'progress': 18}
async_callback: {'progress': 20}
async_callback: {'progress': 22}
async_callback: {'progress': 24}
async_callback: {'progress': 26}
async_callback: {'progress': 28}
async_callback: {'progress': 30}
async_callback: {'progress': 32}
async_callback: {'progress': 34}
async_callback: {'progress': 36}
async_callback: {'progress': 38}
async_callback: {'progress': 40}
async_callback: {'progress': 42}
async_callback: {'progress': 44}
async_callback: {'progress': 46}
async_callback: {'progress': 48}
async_callback: {'progress': 51}
async_callback: {'progress': 53}
async_callback: {'progress': 55}
async_callback: {'progress': 57}
async_callback: {'progress': 59}
async_callback: {'progress': 61}
async_callback

### Step 3. Asking a question

In [5]:
# Retrieve from query
results = await rag.retrieve_async(
    collection_name="demo",
    query_texts="Who are the young seniors?",
    n_results=1
)
for result in results:
    print(result)


{'documents': 'and the Merdeka Generation; “Seniors”, because you will soon retire, or maybe you have recently retired. Young Seniors are in a unique position today. Compared to the Pioneer and Merdeka Generations, you have benefited more from Singapore’s growth, and generally done better in life. But compared to workers younger than you, in their 30s and 40s today, you have generally earned less over your lifetimes. You have also had less time to benefit from improvements to the CPF system, and so have built up less retirement savings. Young Seniors are also in a particularly sandwiched phase of your lives. You have to shoulder the responsibility of caring for both the young and old in your families. Your kids may be young adults, but often are not yet fully independent and still live in the same household. Many of you Young Seniors also have elderly parents at home, who may be beset with the infirmities of old age. You have to shuttle them to medical appointments and hospital visits,

### 3. List Collections

In [6]:
await rag.list_collections_async()

{'collections': ['demo']}

### 4. List Document Headers

In [7]:
docs = await rag.list_documents_async(collection_name="demo")
docs

[IndexedDocPydantic(Id='7eJgbvZr6HtO7lPIhPto2FF0QbWtwaQc-9TleTFFS5I', CollectionName='demo', ByteSize=44189, FileName='43731eec-fd2d-4d4f-9d75-4e9afb8ddd3b.txt', FileType='txt', File=None, Source='https://www.pmo.gov.sg/Newsroom/national-day-rally-2023', Abstract=None, Authors='', Title='2023 National Day Rally Speech', Publisher='', PublishedDate=None, Comments='', Keywords='', CreatedAt=datetime.datetime(2025, 1, 15, 1, 15, 51, 624810), UpdatedAt=datetime.datetime(2025, 1, 15, 1, 15, 51, 624819), IsActive=True, ChunkGroups=[IndexedDocChunkGroupPydantic(Id='61b16a48-6759-43ce-928e-31b954d5a28f', DocumentId='7eJgbvZr6HtO7lPIhPto2FF0QbWtwaQc-9TleTFFS5I', SplitAlgo='recursive_split', ChunkCount=49, ChunkSize=1000, Overlap=100, IsActive=True, ChunksDir='/tmp/chunks/1c08f264366c4f5e981ffbe0b6bdfdda')])]

### 5. Download File

In [8]:
await rag.get_document_file_async(collection_name="demo",document_id=docs[0].Id)

---

## Appendix: Multi-Step Indexing

This achieves the same outcome as step 2 but broken down into multiple steps for ease of debugging. For this demo, we shall use a PDF file.

### Step 0: Setup

In [1]:
import os
from gai.rag.client.rag_client_async import RagClientAsync

rag = RagClientAsync(config={
    "client_type": "gai",
    "url": "http://localhost:12036/gen/v1/rag",
    "extra":{
        "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws"
    }
})



### Step 1: Create header

In [2]:
here = os.path.dirname(__name__)
file_path=os.path.join(here,"ReAct-2210.03629.pdf")
doc_header = await rag.step_header_async(
    collection_name="demo",
    file_path=file_path,
    title="ReAct: Synergizing Reasoning and Acting in Language Models",
    source="https://arxiv.org/abs/2210.03629",
)
doc_header

IndexedDocPydantic(Id='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', CollectionName='demo', ByteSize=633805, FileName='ReAct-2210.03629.pdf', FileType='.pdf', File=None, Source='https://arxiv.org/abs/2210.03629', Abstract=None, Authors='', Title='ReAct: Synergizing Reasoning and Acting in Language Models', Publisher='', PublishedDate=None, Comments='', Keywords='', CreatedAt=datetime.datetime(2025, 1, 15, 1, 17, 39, 995269), UpdatedAt=datetime.datetime(2025, 1, 15, 1, 25, 4, 893489), IsActive=True, ChunkGroups=[IndexedDocChunkGroupPydantic(Id='96c2c87f-97c5-4587-a68b-d36862043a14', DocumentId='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', SplitAlgo='recursive_split', ChunkCount=114, ChunkSize=1000, Overlap=100, IsActive=True, ChunksDir='/tmp/chunks/494a1410dbff466fae862532b852914b')])

### Step 2: Split Document

In [11]:
chunkgroup = await rag.step_split_async(
    collection_name="demo",
    document_id=doc_header.Id,
    chunk_size=1000,
    chunk_overlap=100
)
chunkgroup

IndexedDocChunkGroupPydantic(Id='96c2c87f-97c5-4587-a68b-d36862043a14', DocumentId='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', SplitAlgo='recursive_split', ChunkCount=114, ChunkSize=1000, Overlap=100, IsActive=True, ChunksDir='/tmp/chunks/494a1410dbff466fae862532b852914b')

### Step 3: Index Chunks

In [12]:
# Create callback
import json
async def async_callback(text):
    jsoned = json.loads(text)
    print(f"async_callback: {jsoned}")

# Spin off listener
from gai.lib.common.StatusListener import StatusListener
ws_url = f"ws://localhost:12036/gen/v1/rag/index-file/ws/demo"
listener = StatusListener(ws_url)
import asyncio
listen_task=asyncio.create_task(listener.listen(async_callback))

result = await rag.step_index_async(
    collection_name="demo",
    document_id=chunkgroup.DocumentId,
    chunkgroup_id=chunkgroup.Id,
    async_callback=async_callback
)
result


async_callback: {'progress': 0}
async_callback: {'progress': 1}
async_callback: {'progress': 2}
async_callback: {'progress': 3}
async_callback: {'progress': 4}
async_callback: {'progress': 5}
async_callback: {'progress': 6}
async_callback: {'progress': 7}
async_callback: {'progress': 7}
async_callback: {'progress': 8}
async_callback: {'progress': 9}
async_callback: {'progress': 10}
async_callback: {'progress': 11}
async_callback: {'progress': 12}
async_callback: {'progress': 13}
async_callback: {'progress': 14}
async_callback: {'progress': 14}
async_callback: {'progress': 15}
async_callback: {'progress': 16}
async_callback: {'progress': 17}
async_callback: {'progress': 18}
async_callback: {'progress': 19}
async_callback: {'progress': 20}
async_callback: {'progress': 21}
async_callback: {'progress': 21}
async_callback: {'progress': 22}
async_callback: {'progress': 23}
async_callback: {'progress': 24}
async_callback: {'progress': 25}
async_callback: {'progress': 26}
async_callback: {'pro

IndexedDocChunkIdsPydantic(DocumentId='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', ChunkgroupId='96c2c87f-97c5-4587-a68b-d36862043a14', ChunkIds=['86043838-d64b-434b-98de-5eebae2ef11e', '6c4165e6-1623-43d4-a5c0-3ca17be10187', 'ff6e5eae-0077-4999-95c4-f4c74730385b', 'fc29140a-372a-4c5f-80f2-d286d14ea478', 'd06c0e07-d9bc-4535-a114-254275c68fef', '0e136f46-2c54-45ef-9819-237deff7f66f', '38ef9364-b3e4-4222-9906-52ee9d4f1732', 'aeec1790-dab3-410a-b1b9-2ab4b5c4477e', 'e6ba1a37-b3da-43ff-bb8e-0c4ef856ba4c', '9276b850-a2f5-4524-9565-063416f7c2f7', '90497f69-c1f0-4395-bb66-07467dfe6049', '0d72fb8a-484c-4ba7-a971-ce46e3b57bdf', '53f4fb24-fef6-4d12-bec9-aa932dc140f4', 'a49fbde2-7bd7-46bc-be68-6d2395e701c6', '3b8e2bbb-6b93-41d2-9b56-84aa24aafe68', 'e64f1ff2-861e-455c-9f8a-604a930c2d6d', 'baa5f5b2-619e-4638-9398-a576f77cf2ff', 'ea8a97f0-90ee-411c-b48c-d43853c8a3f2', 'ffd09b8a-2a58-480f-870a-71dc7a132a3e', 'e6d9e0b1-0793-43ae-ab46-5c63eab61472', '39d7187f-f443-4043-8f80-9cdc1dd07045', '5aec2ceb-c9